In [1]:
pip install opencv-python numpy



   ---------------------------------------- 0.0/39.5 MB ? eta -:--:--
    --------------------------------------- 0.8/39.5 MB 11.2 MB/s eta 0:00:04
   - -------------------------------------- 1.3/39.5 MB 6.1 MB/s eta 0:00:07
   -- ------------------------------------- 2.6/39.5 MB 4.7 MB/s eta 0:00:08
   -- ------------------------------------- 2.6/39.5 MB 4.7 MB/s eta 0:00:08
   --- ------------------------------------ 3.9/39.5 MB 4.1 MB/s eta 0:00:09
   ---- ----------------------------------- 4.2/39.5 MB 4.1 MB/s eta 0:00:09
   ----- ---------------------------------- 5.2/39.5 MB 3.8 MB/s eta 0:00:10
   ----- ---------------------------------- 5.8/39.5 MB 3.8 MB/s eta 0:00:09
   ------ --------------------------------- 6.8/39.5 MB 3.7 MB/s eta 0:00:09
   ------- -------------------------------- 7.6/39.5 MB 3.8 MB/s eta 0:00:09
   ------- -------------------------------- 7.9/39.5 MB 3.7 MB/s eta 0:00:09
   -------- ------------------------------- 8.1/39.5 MB 3.4 MB/s eta 0:00:10
   -

In [8]:
import cv2
import numpy as np
import urllib.request

# Download deploy.prototxt
url_prototxt = "https://raw.githubusercontent.com/chuanqi305/MobileNet-SSD/master/deploy.prototxt"
urllib.request.urlretrieve(url_prototxt, "deploy.prototxt")

# Download model weights
url_model = "https://github.com/chuanqi305/MobileNet-SSD/raw/master/mobilenet_iter_73000.caffemodel"
urllib.request.urlretrieve(url_model, "mobilenet_iter_73000.caffemodel")

print("Files downloaded successfully.")


# Load the pre-trained MobileNet SSD model
net = cv2.dnn.readNetFromCaffe("deploy.prototxt", "mobilenet_iter_73000.caffemodel")

# Object classes
CLASSES = ["background", "aeroplane", "bicycle", "bird", "boat",
           "bottle", "bus", "car", "cat", "chair", "cow", "diningtable",
           "dog", "horse", "motorbike", "person", "pottedplant", "sheep",
           "sofa", "train", "tvmonitor"]

# Initialize webcam
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("Error: Could not open webcam.")
    exit()
    
# Setup VideoWriter to save output
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
fourcc = cv2.VideoWriter_fourcc(*'XVID')  # Codec
out = cv2.VideoWriter('output.avi', fourcc, 20.0, (frame_width, frame_height))

# Frame count logic
frame_count = 0
MAX_FRAMES = 300

while True:
    ret, frame = cap.read()
    if not ret:
        break
    frame_count += 1

    (h, w) = frame.shape[:2]
    blob = cv2.dnn.blobFromImage(cv2.resize(frame, (300, 300)),
                                 0.007843, (300, 300), 127.5)

    net.setInput(blob)
    detections = net.forward()

    # Initialize object count dictionary for this frame
    object_counts = {}

    # Process each detection
    for i in range(detections.shape[2]):
        confidence = detections[0, 0, i, 2]

        if confidence > 0.5:
            idx = int(detections[0, 0, i, 1])
            label_name = CLASSES[idx]

            # Count the object
            object_counts[label_name] = object_counts.get(label_name, 0) + 1

            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            (startX, startY, endX, endY) = box.astype("int")

            label = f"{label_name}: {confidence*100:.1f}%"
            cv2.rectangle(frame, (startX, startY), (endX, endY),
                          (0, 255, 0), 2)
            y = startY - 15 if startY - 15 > 15 else startY + 15
            cv2.putText(frame, label, (startX, y),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)

    # Display object counts on top-left of the frame
    y_offset = 20
    for obj, count in object_counts.items():
        count_label = f"{obj}: {count}"
        cv2.putText(frame, count_label, (10, y_offset),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)
        y_offset += 25

    # Write frame to output video
    out.write(frame)
    # Show output
    cv2.imshow("Real-Time Object Detection", frame)

    # Exit on 'q' or after MAX_FRAMES
    if cv2.waitKey(1) & 0xFF == ord("q") or frame_count >= MAX_FRAMES:
        break
        
# Show final frame and wait
cv2.imshow("Final Frame - Press any key to exit", frame)
cv2.waitKey(0)

# Cleanup
cap.release()
out.release()
cv2.destroyAllWindows()
import os
os.startfile("output.avi")  # For Windows



Files downloaded successfully.


In [ ]:
s